<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Parámetros" data-toc-modified-id="Parámetros-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Parámetros</a></span></li><li><span><a href="#Estaciones" data-toc-modified-id="Estaciones-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Estaciones</a></span></li><li><span><a href="#Obtener-todos-los-registros-de-mediciones" data-toc-modified-id="Obtener-todos-los-registros-de-mediciones-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Obtener todos los registros de mediciones</a></span><ul class="toc-item"><li><span><a href="#Obtener-jsons" data-toc-modified-id="Obtener-jsons-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Obtener jsons</a></span></li><li><span><a href="#Formatear-json-a-csv" data-toc-modified-id="Formatear-json-a-csv-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Formatear json a csv</a></span></li></ul></li></ul></div>

In [1]:
# Vamos a utilizar estas librerías
import pandas as pd
import requests
import json
import numpy as np
from datosgobmx import client # https://github.com/mxabierto/api_python_client
import mplleaflet

%load_ext autoreload
%autoreload 2
%matplotlib inline

# Parámetros

In [2]:
# Obtener los parámetros válidos para la API usando el endpoint de Sinaica
parametros_request = client.makeCall('sinaica-parametros')

valid_params = []

for v in parametros_request['results']:
    aux = pd.DataFrame.from_dict(v, orient='index').T
    valid_params.append(aux)
    
valid_params = pd.concat(valid_params, ignore_index=True)
valid_params['date-insert'] = pd.to_datetime(valid_params['date-insert'])
valid_params

CALL: https://api.datos.gob.mx/v2/sinaica-parametros


,_id,parametro,date-insert
0,5ba1176fe2705c1932aa83e7,PM10,2018-09-18 15:19:11.304
1,5ba1176fe2705c1932aa83e8,SO2,2018-09-18 15:19:11.304
2,5ba1176fe2705c1932aa83e9,CO,2018-09-18 15:19:11.304
3,5ba1176fe2705c1932aa83ea,O3,2018-09-18 15:19:11.304
4,5ba1176fe2705c1932aa83eb,NO2,2018-09-18 15:19:11.304
5,5ba1176fe2705c1932aa83ec,TMP,2018-09-18 15:19:11.304
6,5ba1176fe2705c1932aa83ed,PM2.5,2018-09-18 15:19:11.304


# Estaciones

In [3]:
estaciones_request = client.makeCall('sinaica-estaciones', {'pageSize':200})

estaciones = []

for v in estaciones_request['results']:
    aux = pd.DataFrame.from_dict(v, orient='index').T
    estaciones.append(aux)
    
estaciones = pd.concat(estaciones, ignore_index=True)
estaciones['date-insert'] = pd.to_datetime(estaciones['date-insert'])
estaciones = estaciones.apply(lambda x: pd.to_numeric(x, errors='ignore') if x.name!='date-insert' else x)

print(estaciones.shape)
estaciones.head()

CALL: https://api.datos.gob.mx/v2/sinaica-estaciones?pageSize=200
(184, 8)


,_id,lat,long,id,nombre,codigo,redesid,date-insert
0,5ba1176fe2705c1932aa83f0,21.873311,-102.320803,31,CBTIS,CBT,30,2018-09-18 15:19:11.373
1,5ba1176fe2705c1932aa83f1,21.846392,-102.288431,32,Secretaría de Medio Ambiente,SMA,30,2018-09-18 15:19:11.373
2,5ba1176fe2705c1932aa83f2,21.883781,-102.295825,33,Centro,CEN,30,2018-09-18 15:19:11.373
3,5ba1176fe2705c1932aa83f3,31.859917,-116.593722,36,Secundaria,SPABC20,31,2018-09-18 15:19:11.373
4,5ba1176fe2705c1932aa83f4,32.631317,-115.444631,38,UABC,SPABC12,32,2018-09-18 15:19:11.373


In [4]:
estaciones.describe()

,lat,long,id,redesid
count,184.000000,184.000000,184.000000,184.000000
mean,21.925474,-99.709699,191.184783,77.635870
std,5.400582,15.759411,112.805159,32.670787
min,0.000000,-117.056111,31.000000,30.000000
25%,19.391528,-103.316076,95.250000,48.000000
50%,20.570912,-100.355139,165.500000,72.000000
75%,25.526816,-99.119272,267.250000,119.000000
max,34.570000,0.000000,427.000000,149.000000


In [5]:
estaciones.id.nunique(), estaciones._id.nunique(), estaciones.lat.nunique(), estaciones.long.nunique()

(184, 184, 178, 177)

In [6]:
# Estaciones son los datos obtenidos de la API, en formato Pandas DataFrame
# Hay estaciones con valores erróneos
estaciones.plot.scatter('long', 'lat', figsize=(8, 8))
mplleaflet.display()

In [7]:
# Guardamos lo datos para utilizarlos más adelante
# estaciones.to_csv('/data_estaciones_sinaica.csv', index=False)

# Obtener todos los registros de mediciones

In [46]:
# Ejemplo
client.makeCall('sinaica', {'pageSize':5000, 'page':1})

CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=1


{'pagination': {'page': 1, 'pageSize': 5000, 'total': 3244509},
 'results': [{'_id': '5b8ef752e2705c1932ce5ee4',
   'city': 'Guadalajara',
   'date': '2018-01-01T01:00:00.000Z',
   'date-insert': '2018-09-04T21:21:21.989Z',
   'estacionesid': 101,
   'fecha': '2018-01-01',
   'hora': 1,
   'id': '101CO18010101',
   'parametro': 'CO',
   'state': 'Jalisco',
   'validoorig': 1,
   'valororig': 1.3865},
  {'_id': '5b8ef752e2705c1932ce5ee5',
   'city': 'Guadalajara',
   'date': '2018-01-01T02:00:00.000Z',
   'date-insert': '2018-09-04T21:21:21.995Z',
   'estacionesid': 101,
   'fecha': '2018-01-01',
   'hora': 2,
   'id': '101CO18010102',
   'parametro': 'CO',
   'state': 'Jalisco',
   'validoorig': 1,
   'valororig': 1.8923},
  {'_id': '5b8ef752e2705c1932ce5ee6',
   'city': 'Guadalajara',
   'date': '2018-01-01T03:00:00.000Z',
   'date-insert': '2018-09-04T21:21:21.995Z',
   'estacionesid': 101,
   'fecha': '2018-01-01',
   'hora': 3,
   'id': '101CO18010103',
   'parametro': 'CO',
   'st

In [23]:
# Número de registros totales/número de registros por página = Número de páginas que necesitamos
3244192/5000

648.8384

## Obtener jsons

In [47]:
from tqdm import tqdm_notebook
from pathlib import Path

# Hay 3243971 registros el 4 de septiembre 2018 a las 15:13
for page in tqdm_notebook(range(1 , 650)):
    
    filename = 'page_%i_sinaica_mediciones.json' %page
    filename = '/mediciones_sinaica_json/'+filename
    # Revisamos si el archivo ya existe (suponemos que no hay cambios en la paginación)
    if not Path(filename).is_file():
    if True:
        data_api = client.makeCall('sinaica', {'pageSize':5000, 'page':page})
        with open(filename, 'w') as outfile:
            json.dump(data_api, outfile)

HBox(children=(IntProgress(value=0, max=451), HTML(value='')))

CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=199
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=200
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=201
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=202
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=203
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=204
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=205
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=206
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=207
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=208
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=209
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=210
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=211
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=212
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=213
CALL: https://api.datos.g

CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=326
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=327
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=328
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=329
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=330
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=331
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=332
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=333
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=334
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=335
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=336
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=337
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=338
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=339
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=340
CALL: https://api.datos.g

CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=453
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=454
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=455
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=456
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=457
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=458
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=459
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=460
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=461
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=462
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=463
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=464
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=465
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=466
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=467
CALL: https://api.datos.g

CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=580
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=581
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=582
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=583
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=584
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=585
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=586
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=587
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=588
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=589
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=590
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=591
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=592
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=593
CALL: https://api.datos.gob.mx/v2/sinaica?pageSize=5000&page=594
CALL: https://api.datos.g

## Formatear json a csv

In [ ]:
import json
from glob import glob
from joblib import Parallel, delayed

def parse_mediciones_json(json_file):
    "Función auxiliar para convertir los jsons de mediciones de calidad de aire en Pandas Data Frame"
    # Abrimos el archivo json descargado previamente
    with open(json_file, 'r') as aux:
        results = json.load(aux)['results'] # Los resultados vienen en la seccion *results*
    
    # Vamos a guardar los datos de resultados en una lista para después concatenarlos en un DataFrame
    pre_data = []
    
    for r in results:
        # Obtenemos DataFrame previo y lo agregamos
        aux = pd.DataFrame.from_dict(r, orient='index').T
        pre_data.append(aux)
    
    # Concatenamos todos los datos y devolvemos el DataFrame, siempre que haya datos
    if len(pre_data)>0:
        pre_data = pd.concat(pre_data, ignore_index=True)
        return pre_data
    
    # Si por alguna razón no hay datos imprimimos el resultado y no devolvemos nada
    else:
        print('NO DATA FOR ', json_file)
        return

In [20]:
name = '/sinaica_mediciones_json/page_%i_sinaica_mediciones.json'

c = 1

for i, j in zip([1, 501], [501, 649]):
    print('Chunk', c)
    aux_list = [name %n for n in range(i, j)]

    sinaica_mediciones = Parallel(n_jobs=-1, verbose=8)(delayed (parse_mediciones_json)(f) for f in aux_list)

    sinaica_mediciones = pd.concat(sinaica_mediciones, ignore_index=True)
    print(sinaica_mediciones.shape)
    sinaica_mediciones.to_csv('/mediciones_sinaica_csv/sinaica_mediciones_%i_%i.csv' %(i, j),
                              index=False)
    c+=1

Chunk 1


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   25.0s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   45.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 205 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  4.0min finished


(2500000, 12)
Chunk 2


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-1)]: Done 148 out of 148 | elapsed:  1.2min finished


(740000, 12)


In [21]:
all_data_mediciones = []
for file in glob('/mediciones_sinaica_csv/*.csv'):
    aux = pd.read_csv(file,
                      usecols = ['estacionesid', 'fecha', 'hora', 'city', 'state',
                                 'parametro', 'valororig', 'validoorig'])
    all_data_mediciones.append(aux)
    
all_data_mediciones = pd.concat(all_data_mediciones)
print(all_data_mediciones.shape)
all_data_mediciones.head()

(3240000, 8)


,city,estacionesid,fecha,hora,parametro,state,validoorig,valororig
0,Durango,58,2018-08-01,13,PM10,Durango,1,988.61
1,Durango,58,2018-08-01,14,PM10,Durango,1,988.61
2,Durango,58,2018-08-01,15,PM10,Durango,1,988.60
3,Durango,58,2018-08-01,16,PM10,Durango,1,988.63
4,Durango,58,2018-08-01,17,PM10,Durango,1,988.63


In [22]:
all_data_mediciones.estacionesid.nunique()

123

In [23]:
all_data_mediciones.hora.value_counts()

9     140201
10    140097
8     139461
11    139188
17    139004
18    138913
16    138733
19    138639
20    138600
12    138533
15    138433
13    138380
21    138138
14    137879
22    137833
7     137731
6     136910
5     136227
4     132581
3     131681
2     124890
0     122954
23    119612
1     115382
Name: hora, dtype: int64

In [24]:
# all_data_mediciones.to_csv('/data_mediciones_todas_estaciones.csv', index=False)